Ideias:

New is always better (Heat map para ver se ando a ouvir musica que adicionei recentemente)

Divisão por estilos

Divisão por artistas

Horas ouvidas por dia

Horas mais frequentes a ouvir

Estilo mais predominante por hora

Caracteristicas da musica

In [2]:
# !pip install spotipy --user
# !pip install wordcloud --user
# !pip install pandas --user
# !pip install numpy --user
# !pip install matplotlib --user
# !pip install seaborn --user
# !pip install git+https://github.com/plamere/spotipy.git --upgrade
# !pip install seaborn --user
# !pip install python-dotenv --user
# !pip install openpyxl --user
# !pip install pyarrow --user

In [5]:
!pip install git+https://github.com/plamere/spotipy.git --upgrade

  Cloning https://github.com/plamere/spotipy.git to c:\users\hugo\appdata\local\temp\pip-req-build-p6leiu8t


  Running command git clone -q https://github.com/plamere/spotipy.git 'C:\Users\Hugo\AppData\Local\Temp\pip-req-build-p6leiu8t'
You should consider upgrading via the 'c:\python39\python.exe -m pip install --upgrade pip' command.


In [3]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth, SpotifyClientCredentials
import pandas as pd
import time
import numpy as np
from functions.spotipyTools import *
from dotenv import load_dotenv
import os
import openpyxl

ModuleNotFoundError: No module named 'spotipy'

In [ ]:
load_dotenv()

True

In [ ]:
e_client_id = os.environ["client_id"]
e_client_key = os.environ["client_key"]
e_playlist_id = os.environ["playlist_id"]
e_user_id = os.environ["user_id"]

client_credentials_manager = SpotifyClientCredentials(e_client_id, e_client_key)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [ ]:
all_tracks, dates_added = get_playlist_info(e_user_id, e_playlist_id, sp)

track_info_list = []

for item in all_tracks:
  track_info_list.append(get_track_info(item, sp))

df_tracks = pd.DataFrame(track_info_list, columns=['name', 'artist_name', 'album_name', 'album_date', 'album_popularity', 'track_duration', 'danceability', 'energy', 'instrumentalness', 'liveness', 'tempo','artist_url'])

df_tracks['date_added'] = dates_added

In [ ]:
#df_tracks['album_year'] = pd.to_datetime(df_tracks['album_year'], format='%Y-%m-%d').dt.year
df_tracks['album_year'] = pd.to_datetime(df_tracks['album_date'], format='%Y-%m-%d').dt.year
df_tracks['date_added'] = df_tracks['date_added'].map(lambda x:pd.to_datetime(x[:10], format='%Y-%m-%d'))

In [ ]:
track_minutes = np.floor(df_tracks['track_duration']/1000/60).astype(int).astype(str)
track_seconds = np.mod(df_tracks['track_duration']/1000,60).astype(int).astype(str)
track_time = track_minutes.append(track_seconds).groupby(level=0).agg(':'.join)
df_tracks['track_duration'] = pd.to_datetime(track_time, format='%M:%S').dt.time
df_tracks['track_duration_secs'] = df_tracks['track_duration'].apply(lambda x: x.minute * 60 + x.second)

In [ ]:
df_tracks.tail()

In [ ]:
df_tracks.to_excel('PlaylistTracks.xlsx', engine='openpyxl')
df_tracks.to_parquet('PlaylistTracks.parquet', engine='pyarrow')